# **Local invariant features**
A great variety of computer vision problems can be dealt by finding the so called **corresponding points**.
> "Correspondances are everything [in computer vision]."

**Corresponding (or homologous) points**: image points that are the projection of the same 3d point in different views of the scene.

Establishing correspondances may be difficult since point may look different due to viewpoint variations and lighting changes.

*Example*: mosaicing. 
It consists in create larger images by aligning two images of the same scene.
The aligning can be done by:
1. Estimating an homography (at least four points, more is better for least square estimation);
1. Find salient points independetly in the two images;
1. Compute a local description $D(p) = f \bigl(\mathcal{N}(p)\bigr)$ to recognize the salient points;
1. Compare descriptions to tell apart salient points (distance minimizing).

<center> <img src=https://i.ibb.co/cgkqKFX/photo-2021-01-25-08-45-02.jpg width="800px" /> </center>

*Note*: we don't compare patches but descriptors, hence, functions of the patches based on the neighbourhood. 
This because patches are sensitive to lighting, rotation, variations, etc...
But we want an **invariant transformation**.

## The local invariant features paradigm
The task of establishing correspondances is split into three successive steps:
1. Detection of the salient points (also called keypoints);
1. Description: computation of a suitable descriptor based on a neighbourhood around the keypoint;
1. Matching descriptors between images.

The correspondances should be invariant (and robust) to the many transformation that may relate images.

### Properties of detectors
* **Repeatability**: it should find the same keypoints in different views, despite transformations undergone by the images;
* **Interestingness or saliency**: it should find keypoints surrounded by informative patterns of intensities, which would render them amenable to be told apart by the matching process (we need to find points and distinguish the neighbourhood too). 

### Properties of descriptors
A descriptors it's a vector-valued function and it's a way to compress informations.
* **Distinctiveness vs. robustness trade-off**: the descriptor should capture the salient information around a keypoint (keep importan tokens) and disregards changes due to nuisances and noise;
* **Compactness**: the descriptor should be as concise as possible to minimize memory occupancy and allow efficient matching (i.e. hamming distance, a very fast binary descriptor).

Speed is desiderable for both, but in particular for detectors since the run over the whole image.
Sometimes detector and descriptor algorithms are merged together (i.e. SIFT + DOG).

## Performance of the matching process
As long as one tries to gather more matches, these become less precise.
Two measures to evaluate the performances are:
* **Recall**: $\frac{TP}{P}$;
* **Sensitivity**: $\frac{TP}{TP+FP}$.

<center> <img src=https://i.ibb.co/Jkd2fHr/photo-2021-01-25-09-11-04.jpg width="800px" /> </center>

## Interesting points vs. edges
Edge pixel can be hardly told apart since they look very similar along the direction orthogonal to the gradient.
Edges are locally ambiguous, while corners are more useful since that pixels exhibiting a **large variations along all directions** (or at least as many as possible) are more amenable to establishing reliable correspondances.
<center> <img src=https://i.ibb.co/88pnMXm/photo-2021-01-25-09-32-08.jpg width="800px" /> </center>

### Moravec interest point detector
A simple index $C = \min \Bigl\{ \sum_{i,j \in w} \bigl( I(i+m,j+n) - I(i,j) \bigr)^2 \Bigr\}$ for $m,n \in \{-1,0,1\} \ne (0,0)$. 
The higher the better.

<center> <img src=https://i.ibb.co/stfBTrw/photo-2021-01-25-09-37-51.jpg width="800px" /> </center>

### Harris corner detector
It's considered the father of corner detectors and it's based on a continuous formulation of the Moravec's error function.
Since it's not discretized, we can consider the infinitesimal shift $(\Delta x, \Delta y)$ and write the function as:
<center> <img src=https://i.ibb.co/XFnHhfc/photo-2021-01-25-09-44-27.jpg width="1000px" /> </center>

*Note*: the weight function makes the global function a local property.

We can deploy Taylor's expansion of the intensity function at $(x,y)$:

$$I(x + \Delta x, y + \Delta y) \approx I(x,y) + \frac{\partial I(x,y)}{\partial x} \Delta x + \frac{\partial I(x,y)}{\partial y} \Delta y$$
$$I(x + \Delta x, y + \Delta y) - I(x,y) \approx \frac{\partial I(x,y)}{\partial x} \Delta x + \frac{\partial I(x,y)}{\partial y} \Delta y = I_x(x,y) \Delta x + I_y(x,y) \Delta y$$

And so, the error function can be rewritten as:
$$E(\Delta x, \Delta y) = \sum_{x,y} w(x,y) \Bigl( I_x(x,y) \Delta x + I_y(x,y) \Delta y \Bigr)^2$$
and developing the binomial square:
$$E(\Delta x, \Delta y) = \sum_{x,y} w(x,y) \Bigl( I_x(x,y)^2 \Delta x^2 + I_y(x,y)^2 \Delta y^2  + 2I_x(x,y) I_y(x,y) \Delta x \Delta y \Bigr)$$

$$ = \sum_{x,y} w(x,y) \begin{pmatrix} {\begin{bmatrix} \Delta x & \Delta y \end{bmatrix}} & {\begin{pmatrix} I_x(x,y)^2 & I_x(x,y) I_y(x,y) \\ I_x(x,y) I_y(x,y) & I_y(x,y)^2 \end{pmatrix}} & {\begin{bmatrix} \Delta x \\ \Delta y \end{bmatrix}} \end{pmatrix} $$

$$ = \begin{bmatrix} \Delta x & \Delta y \end{bmatrix} \begin{pmatrix} \sum_{x,y} w(x,y) I_x(x,y)^2 & \sum_{x,y} w(x,y) I_x(x,y) I_y(x,y) \\ \sum_{x,y} w(x,y) I_x(x,y) I_y(x,y) & \sum_{x,y} w(x,y) I_y(x,y)^2 \end{pmatrix} \begin{bmatrix} \Delta x \\ \Delta y \end{bmatrix} $$

hence: 
$$E(\Delta x, \Delta y) = \begin{bmatrix} \Delta x & \Delta y \end{bmatrix} M \begin{bmatrix} \Delta x \\ \Delta y \end{bmatrix}$$
where $M$ is the structure matrix, that encodes the local image structure around the considered pixel.

Let's hypothesize tha $M$ is a diagonal matrix (all real matrixes can always be diagonalized):
$$M = \begin{pmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{pmatrix}$$
hence:
$$E(\Delta x, \Delta y) = \begin{bmatrix} \Delta x & \Delta y \end{bmatrix} \begin{pmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{pmatrix} \begin{bmatrix} \Delta x \\ \Delta y \end{bmatrix} = \lambda_1 \Delta x^2 + \lambda_2 \Delta y^2$$
then:
* If $\lambda_1, \lambda_2 \approx 0 \to E(\Delta x, \Delta y) \approx 0$ and we'll have a **flat area**;
* If $\lambda_1 >> \lambda_2 \to E(\Delta x, \Delta y) \approx \lambda_1 \Delta x^2$ and we'll have an **edge**;
* If $\lambda_1, \lambda_2 \uparrow \to E(\Delta x, \Delta y) \uparrow$ and we'll have a **corner**.

<center> <img src=https://i.ibb.co/GksDRjn/photo-2021-01-25-10-13-32.jpg width="1000px" /> </center>

The previous considerations have general validity as $M$ is real and symmetric, thus can always be diagonalized by a rotation of the image reference frame (we don't even need to do that, we just need the eigenvalues):
$$M = R \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} R^T$$

The columns of $R$ are the orthogonal unit eigenvectors of $M$ and $\lambda_i$ the corresponding eigenvalues. 
$R^T$ is the rotation matrix that aligns the images reference frame axes to the eigenvectors of $M$.

Compute the eigenvalues of $M$ may be slow, so a way to compute a more efficient cornerness function is to compute:
$$C = \det(M) - k \cdot \text{tr}(M)^2 = \lambda_1 \lambda_2 - k(\lambda_1 + \lambda_2)^2$$

We compute determinant (product of eigenvalues) and trace (sum of eigenvalues), that are less computational expensive.

<center> <img src=https://i.ibb.co/RY4wkjH/photo-2021-01-25-10-20-21.jpg width="1000px" /> </center>

The Harris corner detection algorithm can conceptually be summarized as:
1. Compute $C$ at each pixel;
1. Select al pixel where $C$ is higher than a positive threshold (to tune);
1. In the previous set, detect as corners only those pixels that are local maxima of $C$.

The weightinf function $w(x,y)$ is Gaussian rather than box-shaped, so it assigns more weight to closer pixels and less weight to those farther away.


### Shi-Tomasi corner detector
It's a popular variant of the previous detector that uses a different cornerness function that provides better results when tracking corner points across video frames.

It selects the minimum eigenvalue of the structure matrix: $C = \min(\lambda_1,\lambda_2)$.

<center> <img src=https://i.ibb.co/bJ38qZ5/photo-2021-01-25-10-55-04.jpg width="600px" /> </center>

## Invariance properties
* **There is an invariance to rotation?** Yes, since eigenvalues of $M$ are invariant to a rotation to the image reference frame axes;
* **There is an invariance to intensity changes?** It's limited, in particular:
> * Yes, to an additive bias $I' = I + b$, i.e. due to lighting changes, since derivative suppresses constant terms, while mantains factors;
  * No, to multiplication by a gain factor $I'=a \cdot I$, since derivative get multiplied by the same factor.
<center> <img src=https://i.ibb.co/2Nn6wXd/photo-2021-01-25-11-02-25.jpg width="700px" /> </center>

* **There is an invariance to scale?** The use of a fixed detection window size makes it impossible to repeatably detect homologous features when they appear at a different scales in images:
<center> <img src=https://i.ibb.co/MShbwFX/photo-2021-01-25-11-07-27.jpg width="800px" /> </center>
Typically, an image contains features at different scales (for example, some points are interesting as long as a proper neighbourhood size is chosen, to evaluate the chosen interestingness criterion).

## Scale-space
Thus, detecting all features calls for a tool capable of analyzing the image across the whole range of the scales deemed as relevant (multi-scale analysis tool).
Depending on the acquisition settings (distance and focal lenght) an object may look differently in the image (it may exhibit more or less details or features).
<center> <img src=https://i.ibb.co/yg842CZ/photo-2021-01-25-11-15-06.jpg width="800px" /> </center>

**Features do exist within a certain range of scales**.
The previiusly mentioned tool would also allow for finding homologous features, despite their showing-up at different scales.

The same feature would simply be detected at different steps within a **multi-scale image analysis process**.

The key idea is applying a fixed tool size on scaled and increasingly blurred versions of the input image (we can use a Gaussian filter since it mantains only the rough structure, so the features will look similar). This representation is known as **scale- space**.
<center> <img src=https://i.ibb.co/0QmdcNc/photo-2021-01-25-11-23-11.jpg width="800px" /> </center>

## Gaussian scale-space
A scale-space is a one-parameter family of images, created from the original one, so that the structures at smaller scales are successively suppressed by smooting operations (without creating new structures).

A scale-space must be realized by Gaussian smoothing:
$$L(x,y,\sigma) = G(x,y,\sigma) \ast I(x,y)$$

So, a scale-space is created by repeatedly smoothing the original image with larger and larger Gaussian kernels (or by solving 2d diffusion PDE over time, from the original image).

By itself, Gaussian scale-space is only a tool to represent the input image at different scales, but it doesn't includes any criterion to deduct features nor to select their scale (it's an open research question).

The fundamental research work on multi-scale feature detection and automatic scale selection is by Lindberg (he mixes proofs and intuitions), who proposed to compute a combination of scale-normalized derivatives (to boost the algorithm) of the Gaussian scale-space and find their extrema.

> If a point is interesting in different scales, which one should I use? Where should I describe it?

## Scale-normalize LOG
Between Lindberg's proposed function there is the scale-normalized **Laplacian of Gaussian** (LOG):
$$F(x,y,\sigma) = \sigma^2 \cdot \nabla^2 L(x,y,\sigma) = \sigma^2 \cdot \bigl( \nabla^2 G(x,y,\sigma) \ast I(x,y) \bigr)$$
<center> <img src=https://i.ibb.co/4Tkpbht/photo-2021-01-25-11-51-39.jpg width="800px" /> </center>

Features (blob-like) and scales are detected as **extrema of the scale-normalized LOG** (saliency function):
<center> <img src=https://i.ibb.co/P50zWzQ/photo-2021-01-25-11-55-31.jpg width="600px" /> </center>

### DoG detector (by Lowe)
A way to detect keypoints is seeking for extrema of the DoG (difference of Gaussian) function, across the $(x,y,\sigma)$ domain:
$$DoG(x,y,\sigma) = \bigl( G(x,y,k\sigma) - G(x,y,\sigma) \bigr) \ast I(x,y) = L(x,y,k\sigma) - L(x,y,\sigma)$$

This approach (DoG) provides a computationally efficient approximation of the scale-normalized LOG:
$$G(x,y,k\sigma) - G(x,y,\sigma) \approx (k-1) \sigma^2 \nabla^2 G(x,y,\sigma)$$
<center> <img src=https://i.ibb.co/VCYMhG5/photo-2021-01-25-12-03-28.jpg width="300px" /> </center>
since $(k-1)$ is a constant factor it doesn't influence the extrema location (not a critical choice).
Note that both detector are rotation invariant (circularly symmetric filters) and find blob-like features (still DoG is preferable).

#### DoG computation
<center> <img src=https://i.ibb.co/FnsJjzH/photo-2021-01-25-12-06-07.jpg width="1000px" /> </center>

#### Keypoint detection and tuning
<center> <img src=https://i.ibb.co/M922yB9/photo-2021-01-25-12-10-27.jpg width="1000px" /> </center>

#### Exemplar DoG keypoints
<center> <img src=https://i.ibb.co/KxK8dN2/photo-2021-01-25-12-12-20.jpg width="1000px" /> </center>

## Scale and rotation invariant description
Most keypoint detections follow a similar approach: finding extrema across a stack of images computed while increasing a scale parameter.

Once each keypoint has been extracted, a sorrounding patch is considered to compute its descriptor, that should be **scale and rotation invariant**:
* To achieve scale invariance, the patch is taken from the stack image, i.e. $L(x,y,\sigma)$, that correspond to the **characteristic scale**;
* To achieve rotation invariance, a canonical patch is computed (in this manner, the descriptor can be computed on a canonicaly-oriented patch, a new reference system, different from the image reference frame).

Lowe proposed to compute the canonical orientation of DoG keypoints as follows:
1. Given a keypoint, the magnitude and orientation of the gradient are computed, at each pixel of the associated Gaussian-smoothed image $L$ (prominent gradient direction):
$$m(x,y) = \sqrt{ \Bigl( L(x+1,y) - L(x-1,y) \Bigr)^2 + \Bigl( L(x,y+1) - L(x,y-1) \Bigr)^2 }$$
$$\theta(x,y) = \tan^-1 \Biggl( \frac{L(x,y+1) - L(x,y-1)}{L(x+1,y) - L(x-1,y)} \Biggr)$$
1. Then, an **orientation histogram** (of angular range, with bin size equal to $10°$) is created by accumulating the contribuitions of the pixels belonging to a neighbourhood of the keypoint location;
1. The contribution of each such pixel (to its designed orientation bin) is given by the **gradient magnitude weighted by a Gaussian**, with $\sigma = 1.5 \sigma_s$, with $\sigma_s$ denoting the scale of the keypoint;
1. The characteristic orientation of the keypoint is given by the **highest peak of the orientation histogram**;
1. Moreover, other peaks higher than $0.8$ of the main one will be kept as well. This means that a keypoint may have multiple canonical orientations and, in turn, multiple descriptors share the same location/scale with different orientations (quite rare).
<center> <img src=https://i.ibb.co/Qd2GxT6/photo-2021-01-25-12-36-36.jpg width="800px" /> </center>
1. Finally, a parabola is fit in the neighbourhood of each peak to achieve a more accurate estimation of the canonical orientation (two adjacent bins to the found peak are considered).

## SIFT descriptor
SIFT (scale invariant feature transform) is a descriptor computed as follows:
1. A 16x16 oriented pixel grid around each keypoint is considered;
1. Each grid is further divided in 4x4 regions (each of size 4x4 pixels);
1. A gradient orientation histogram is created for each region: each histogram has 8 bins (bin size of $45°$);
1. Gradient are rotated according to the canonical orientation of the keypoint;
1. Each pixel in the region contributes to its designated bin, according to gradient magnitude as well as to a Gaussian weighting function centered at the keypoint (with $\sigma$ equal to hald the grid size).

<center> <img src=https://i.ibb.co/T0Z6XDm/photo-2021-01-25-12-47-49.jpg width="1000px" /> </center>

SIFT is biologically inspired.
There are some studies suggesting that neurons in the primary visual cortex (V1) matches gradient orientations robustly wrt a certain degree of shift of the input pattern, for recognition purposes.

## Matching process
Descriptors are compared across different views of a scene, to find corresponding point: it's a classical **nearest-neighbours search** problem:
> Given a set $S$ of points in a metric space $M$, and a query point $q \in M$, find the closest point in $S$ to $q$.

*Note*: in the case of SIFT descriptor, close enough in the SIFT space, so a distance in $\mathbb{R}^{128}$.

Without loss of generality, we assume that we want to match the local features computed at run-time from an image under analysis, called target image $T$, to those already computed from a (set of) reference image(s) $R$:
* The features in $T$ represents the query points $q$;
* The features in $R$ provide the set $S$;
* The metric space $M$ is the descriptor space, endowed by a distance function.

The found NN doesn't necessarily provide a vali correspondence, since some feature in $T$ may not have a corresponding feature in $R$ (generally due to clutters or viewpoint changes).
Hence we have to enforce the criteria to accept or reject a match found by the NN process (use of heuristics). 
Two such criteria are:
1. NN distance (it's a threshold) $d_{NN} \le T$;
2. Ratio between closest and 2nd closest points $\frac{d_{NN}}{d_{2-NN}} \le T$.

*Note*: Lowe shows that $T=0.8$ may allow for rejecting 90% of wrong matches while missing only 5% of correct ones.

Exhaustively searching for the NN of the query feature $q$ has linear complexity in the size of $S$ (very slow). 
Thus, efficient indexing techniques (from database managmente) are deployed to speed-up the process.